In [1]:
# For Gaussian(0,1) ep and Gaussian(0,1) Z

In [3]:
import mpmath as mp

In [4]:
# ---- Core stats for Uniform(0,1) ----
def uniform_LDP_stats(x):
    """
    Return theta_x, I_x, sqrtLpp, C_x for Uniform(0,1) at point x in (0,1).
    """
    if not (0 < x < 1):
        raise ValueError("x must be strictly between 0 and 1.")

    # log-mgf Λ(t) = log((e^t - 1)/t)
    Lambda   = lambda t: mp.log((mp.e**t - 1)/t)
    LambdaP  = lambda t: (mp.e**t*(t-1) + 1)/(t*(mp.e**t - 1))
    # Λ''(t) = 1/t^2 - e^t/(e^t - 1)^2
    LambdaPP = lambda t: 1/t**2 - mp.e**t/((mp.e**t - 1)**2)

    # Special case x=0.5 → theta=0
    if abs(x - 0.5) < 1e-12:
        theta_x = 0.0
        I_x     = 0.0
        sqrtLpp = mp.sqrt(1/12)   # variance of Uniform(0,1)
        C_x     = 0.0
        return theta_x, I_x, sqrtLpp, C_x

    # Solve Λ'(t)=x for t (positive if x>0.5, negative if x<0.5)
    f = lambda t: LambdaP(t) - x
    theta_x = mp.findroot(f, 1 if x > 0.5 else -1)

    # Rate function
    I_x = theta_x*x - Lambda(theta_x)

    # Curvature
    Lpp = 1/12 if abs(theta_x) < 1e-8 else LambdaPP(theta_x)
    sqrtLpp = mp.sqrt(Lpp)

    # C_x = 1 - t/(e^t - 1)
    C_x = 1 - theta_x/(mp.e**theta_x - 1)

    return theta_x, I_x, sqrtLpp, C_x

In [5]:
# ---- Your asymptotic tail probability (right tail, x>0.5) ----
def prob_expression(x, n):
    """
    Your full prefactor * exp{-n I(x)} * n^{-3/4} * (log n)^(-3/8).
    """
    theta_x, I_x, sqrtLpp, C_x = uniform_LDP_stats(x)
    a = 1/(theta_x * sqrtLpp)
    b = 1/mp.sqrt(2)
    c = (mp.sqrt(2)*0.5)**(0.5**2)  # = (sqrt(2)*0.5)^0.25
    exp_inner = -( mp.log(2*mp.pi) + 0.5**2
                   + 0.5**2 * mp.log( C_x/(0.5*mp.sqrt(2*mp.pi)) ) )
    d = mp.e**(exp_inner)
    e = mp.e**(-I_x * n)
    f = n**(-(0.5 + 0.5**2))                 # n^(-0.75)
    g = (mp.log(n))**((0.5**2)/2 - 0.5)      # (log n)^(-0.375)
    return a*b*c*d*e*f*g

In [6]:
# ---- Solve for x given n and alpha (right-tail) ----
def solve_x_from_alpha(n, alpha, seed1=0.5001, seed2=0.9999, tol=1e-10, maxiter=60):
    """Solve Prob(x;n) = 1 - alpha for right tail (0.5<x<1)."""
    target = 1 - alpha
    f = lambda x: prob_expression(x, n) - target

    # Try real-only secant using two seeds
    try:
        x_star = mp.findroot(f, seed1, seed2)
        if abs(mp.im(x_star)) < 1e-12 and 0 < mp.re(x_star) < 1:
            x_star = float(mp.re(x_star))
        else:
            raise ValueError("Non-real or out-of-range iterate")
    except Exception:
        # Robust fallback: bisection on a monotone function (Prob decreases in x>0.5)
        a, b = 0.5001, 0.999
        fa, fb = f(a), f(b)
        if not (fa >= 0 and fb <= 0):
            raise RuntimeError(
                f"Target not bracketed: Prob(a)={fa+target:.3e}, Prob(b)={fb+target:.3e}. "
                f"Try other bounds or check alpha/n."
            )
        for _ in range(maxiter):
            m = 0.5*(a+b)
            fm = f(m)
            if abs(fm) < tol:
                x_star = m
                break
            if fm > 0:  # Prob(m) > target -> need larger x to decrease Prob
                a = m
            else:
                b = m
        else:
            x_star = 0.5*(a+b)

    theta_x, I_x, sqrtLpp, C_x = uniform_LDP_stats(x_star)
    psi = 1/(theta_x*sqrtLpp)   
    return x_star, theta_x, I_x, sqrtLpp, C_x, psi


In [34]:
def ES(x, n, alpha):
    """
    Your full prefactor * exp{-n I(x)} * n^{-3/4} * (log n)^(-3/8).
    """
    theta_x, I_x, sqrtLpp, C_x = uniform_LDP_stats(x)
    a = 1/(theta_x * sqrtLpp)
    b = 1/mp.sqrt(2)
    c = (mp.sqrt(2)*0.5)**(0.5**2)  # = (sqrt(2)*0.5)^0.25
    exp_inner = -( mp.log(2*mp.pi) + 0.5**2
                   + 0.5**2 * mp.log( C_x/(0.5*mp.sqrt(2*mp.pi)) ) )
    d = mp.e**(exp_inner)
    e = mp.e**(-I_x * n)
    f = n**(-(0.5 + 0.5**2))                 # n^(-0.75)
    g = (mp.log(n))**((0.5**2)/2 - 0.5)      # (log n)^(-0.375)
    h = theta_x                       # (Lambda^*)'(x)
    # return x + (a*b*c*d*e*f*g)/( (1-alpha) * n * h )
    return x + 1/( n * h )

In [99]:
n = 1000
alpha = 0.99   # corresponds to Prob ~ 1-alpha
x, theta_x, I_x, sqrtLpp, C_x, psi = solve_x_from_alpha(n, alpha)
es = ES(x, n, alpha)

In [100]:
print("VAR     =", x)
print("ES      =", es)
print("theta_x =", theta_x)
print("I(x)    =", I_x)
print("sqrtLpp =", sqrtLpp)
print("C_x     =", C_x)
print("psi     =", psi)

VAR     = 0.5088025469879969
ES      = 0.518267743235533
theta_x = 0.105650213038141
I(x)    = 0.000464952237354939
sqrtLpp = 0.288594604466914
C_x     = 0.0518951155545102
psi     = 32.7975509626037


In [101]:
# For Gaussian(0,1) ep and Pareto(x_m=1,alpha=2) Z

In [102]:
def prob_expression_2(x, n):
    """
    Your full prefactor * exp{-n I(x)} * n^{-1/2} * (log n)^(-3/2) * C
    """
    theta_x, I_x, sqrtLpp, C_x = uniform_LDP_stats(x)
    a = mp.e**(-I_x * n)
    b = n**(-1/2)
    c = mp.log(n)**(-3/2)
    d = 1/(theta_x * sqrtLpp) * 2 * (2*0.5**2)**(-(2+1)/2) * 0.5 * (2+1)**(-1/2)

    return a*b*c*d

In [103]:
# ---- Solve for x given n and alpha (right-tail) ----
def solve_x_from_alpha_2(n, alpha, seed1=0.5001, seed2=0.9999, tol=1e-10, maxiter=60):
    """Solve Prob(x;n) = 1 - alpha for right tail (0.5<x<1)."""
    target = 1 - alpha
    f = lambda x: prob_expression_2(x, n) - target

    # Try real-only secant using two seeds
    try:
        x_star = mp.findroot(f, seed1, seed2)
        if abs(mp.im(x_star)) < 1e-12 and 0 < mp.re(x_star) < 1:
            x_star = float(mp.re(x_star))
        else:
            raise ValueError("Non-real or out-of-range iterate")
    except Exception:
        # Robust fallback: bisection on a monotone function (Prob decreases in x>0.5)
        a, b = 0.5001, 0.999
        fa, fb = f(a), f(b)
        if not (fa >= 0 and fb <= 0):
            raise RuntimeError(
                f"Target not bracketed: Prob(a)={fa+target:.3e}, Prob(b)={fb+target:.3e}. "
                f"Try other bounds or check alpha/n."
            )
        for _ in range(maxiter):
            m = 0.5*(a+b)
            fm = f(m)
            if abs(fm) < tol:
                x_star = m
                break
            if fm > 0:  # Prob(m) > target -> need larger x to decrease Prob
                a = m
            else:
                b = m
        else:
            x_star = 0.5*(a+b)

    theta_x, I_x, sqrtLpp, C_x = uniform_LDP_stats(x_star)
    psi = 1/(theta_x*sqrtLpp)   
    return x_star, theta_x, I_x, sqrtLpp, C_x, psi


In [104]:
def ES_2(x, n, alpha):
    """
    Your full prefactor * exp{-n I(x)} * n^{-3/4} * (log n)^(-3/8).
    """
    theta_x, I_x, sqrtLpp, C_x = uniform_LDP_stats(x)
    a = mp.e**(-I_x * n)
    b = n**(-1/2)
    c = mp.log(n)**(-3/2)
    d = 1/(theta_x * sqrtLpp) * 2 * (2*0.5**2)**(-(2+1)/2) * 0.5 * (2+1)**(-1/2)
    h = theta_x                       # (Lambda^*)'(x)
    return x + a*b*c*d/( (1-alpha) * n * h )

In [135]:
n = 1000
alpha = 0.999   # corresponds to Prob ~ 1-alpha
x, theta_x, I_x, sqrtLpp, C_x, psi = solve_x_from_alpha_2(n, alpha)
es = ES_2(x, n, alpha)

In [136]:
print("VAR     =", x)
print("ES      =", es)
print("theta_x =", theta_x)
print("I(x)    =", I_x)
print("sqrtLpp =", sqrtLpp)
print("C_x     =", C_x)
print("psi     =", psi)

VAR     = 0.5242260844593866
ES      = 0.527661053598525
theta_x = 0.29112341104497
I(x)    = 0.00352390275416775
sqrtLpp = 0.288064889664327
C_x     = 0.138508925178405
psi     = 11.9242903034406
